## 读取数据

In [1]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
import seaborn as sns
import matplotlib
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
plt.rcParams['figure.dpi'] = 150 # 修改图片分辨率
plt.rcParams.update({'font.size': 18})
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Helvetica Neue']
import matplotlib.pyplot as plt


In [2]:
# Load the data
data = pd.read_excel('./P0022023-study2.xlsx')

# 删除 'Cluster' 列中等于 3 的行
data= data[data['Label'] != 3]

data
exclusive = ['Label']
data[ exclusive] = data[ exclusive].fillna(-1)
data = data.drop(columns=['ID'])
data # display the first few rows of the processed data

,Subtype I Baseline,Subtype II Baseline,Label,Group
0,0.313884,0.686116,1.0,0
1,0.999335,0.000665,0.0,1
2,0.000348,0.999652,1.0,0
3,0.033688,0.966312,1.0,0
4,0.897114,0.102886,0.0,1
...,...,...,...,...
81,0.003672,0.996328,1.0,0
82,0.000000,1.000000,1.0,0
83,0.169101,0.830899,1.0,0
84,0.003777,0.996223,1.0,0


## 数据不平衡数据

In [3]:
df=data
df[exclusive] = df[exclusive].fillna(-1)
cols=[i for i in df.columns if i not in ['Group']]
X=df[cols]
y=df['Group']

from collections import Counter
print(sorted(Counter(y).items()))
from imblearn.over_sampling import SMOTE, ADASYN

X_resampled, y_resampled = SMOTE(sampling_strategy=0.80, random_state=1).fit_resample(X, y)
# 打印处理之前的类别人数
print("处理之前的类别人数:")
print(y.value_counts())

[(0, 57), (1, 27)]
处理之前的类别人数:
0    57
1    27
Name: Group, dtype: int64


In [4]:
from collections import Counter
print(sorted(Counter(y_resampled).items()))

[(0, 57), (1, 45)]


In [5]:
import pickle
# 保存这些变量到文件：X_resampled，y_resampled和cols_select
with open('data/variables-huafen.pkl', 'wb') as file:
    pickle.dump((X_resampled, y_resampled), file)


# 将数据保存为 Excel 文件
if isinstance(X_resampled, pd.DataFrame) or isinstance(X_resampled, pd.Series):
    X_resampled.to_excel('table/X_resampled.xlsx')

if isinstance(y_resampled, pd.DataFrame) or isinstance(y_resampled, pd.Series):
    y_resampled.to_excel('table/y_resampled.xlsx')

In [6]:
X_resampled

,Subtype I Baseline,Subtype II Baseline,Label
0,0.313884,0.686116,1.0
1,0.999335,0.000665,0.0
2,0.000348,0.999652,1.0
3,0.033688,0.966312,1.0
4,0.897114,0.102886,0.0
...,...,...,...
97,0.990126,0.009874,0.0
98,0.927579,0.072421,0.0
99,0.960749,0.039251,0.0
100,0.968768,0.031232,0.0


### 模拟输入

In [7]:
import pickle
from sklearn.model_selection import train_test_split
exclusive = ['Label']
with open('./data/variables-huafen.pkl', 'rb') as file:
    X_resampled, y_resampled = pickle.load(file)
X , y = X_resampled,y_resampled
X

,Subtype I Baseline,Subtype II Baseline,Label
0,0.313884,0.686116,1.0
1,0.999335,0.000665,0.0
2,0.000348,0.999652,1.0
3,0.033688,0.966312,1.0
4,0.897114,0.102886,0.0
...,...,...,...
97,0.990126,0.009874,0.0
98,0.927579,0.072421,0.0
99,0.960749,0.039251,0.0
100,0.968768,0.031232,0.0


In [8]:
_ = 0 
# 步骤1: 分层随机分割数据集
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)
# # 获取划分后的索引
train_indices = X_train.index
test_indices = X_holdout.index
np.savez('split/indices.npz', train=train_indices, test=test_indices)


In [9]:
import numpy as np

data = np.load('./split/indices.npz')

In [10]:
train_PANSS = X_resampled[exclusive].loc[train_indices]
train_PANSS[(train_PANSS != -1).any(axis=1)]

,Label
99,0.0
49,1.0
65,1.0
19,0.0
42,1.0
...,...
21,0.0
26,0.0
87,0.0
59,1.0


In [11]:
test_PANSS = X_resampled[exclusive].loc[test_indices]
print(len(test_PANSS[(test_PANSS != -1).any(axis=1)]))
test_PANSS[(test_PANSS != -1).any(axis=1)]

20


,Label
47,0.0
4,0.0
28,0.0
10,0.0
97,0.0
30,0.0
93,1.0
74,1.0
88,1.0
95,0.0


In [12]:
test_indices

Int64Index([47, 4, 28, 10, 97, 30, 93, 74, 88, 95, 5, 64, 79, 80, 41, 70, 14,
            35, 0, 60, 16],
           dtype='int64')

In [13]:
print("预处理全部完成！")

预处理全部完成！
